In [ ]:
nums = list(range(0, 1000001))

# Parallelize
nums_rdd = sc.parallelize(nums) # this distributes the data and returns an RDD to nums_rdd

# Collect
nums_rdd.collect() 

# Take
nums_rdd.take(5)

# Map
squared_nums_rdd = nums_rdd.map(lambda x: x ** 2)
pairs = nums_rdd.map(lambda x: (x, len(str(x))))
pairs.take(25) # will show us that this worked

# Filter
even_pairs = pairs.filter(lambda x: (x[1] % 2) == 0)
even_pairs.take(25)




